In [39]:
%matplotlib inline

In [40]:
import urllib2
import os
import zipfile
import pandas as pd
import json
import ast
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# download data
def retrieveData():
    url = 'http://crowdcrafting.org/project/landfill/tasks/export?type=task_run&format=csv'
    response = urllib2.urlopen(url)
    output = open('landfill_data.zip', 'wb')
    output.write(response.read())
    output.close()    
    return unzip()

def unzip():
    fh = open('landfill_data.zip', 'rb')
    z = zipfile.ZipFile(fh)
    for name in z.namelist():
        outpath = ""
        z.extract(name, outpath)
    fh.close()
    os.remove('landfill_data.zip')
    
    return load_data(name)

In [3]:
data = pd.read_csv('../landfill_task_run.csv', error_bad_lines=False)

Skipping line 642: expected 20 fields, saw 21



In [4]:
data.shape

(1882, 20)

In [23]:
data[data['task_runinfo__area'] != '0'].shape

(748, 20)

In [27]:
len(data.groupby('task_runinfo__task_id').size())

1235

In [21]:
data[data['task_runinfo__area'] != '0']['task_runinfo__area'].map(lambda x: len(ast.literal_eval(x)))

0        1
1        1
2        1
5        1
6        1
7        2
9        1
11       1
12       1
14       1
17       1
19       1
20       1
21       1
22       1
24       1
26       2
28       1
30       1
31       1
37       1
38       1
39       1
40       1
45       1
46       1
47       1
49       1
51       1
53       2
        ..
1732     2
1735     1
1739     1
1741     1
1744     1
1747     1
1748     1
1754     1
1755     1
1760     1
1767     0
1772     2
1781     1
1788     1
1795     1
1808     1
1810     0
1812     1
1827     1
1834     2
1835     1
1838     1
1841    10
1848     2
1854     1
1856     1
1861     1
1875     1
1877     2
1880     1
Name: task_runinfo__area, dtype: int64

In [22]:
ast.literal_eval(data[data['task_runinfo__area'] != '0']['task_runinfo__area'][7])

[{u'area': 145322.84045764743,
  u'area_geometry': u'{"type":"Polygon","coordinates":[[[-72.48043227249183,42.355712710567595],[-72.48279261642497,42.35647382812253],[-72.48442339950557,42.35412701938756],[-72.48418736511287,42.353619589760605],[-72.48431611114559,42.35308044129209],[-72.48502421432543,42.35266814816614],[-72.48528170639084,42.351811838417035],[-72.48457360321099,42.35084451151944],[-72.48292136245767,42.35157397252739],[-72.48133349472073,42.35365130423191],[-72.48032498413093,42.35384159072626],[-72.47972416931195,42.354206304896984],[-72.47978854232831,42.355712710567595],[-72.48043227249183,42.355712710567595]]]}',
  u'id': u'OpenLayers_Feature_Vector_571'},
 {u'area': 104889.54808553384,
  u'area_geometry': u'{"type":"Polygon","coordinates":[[[-72.47948813491927,42.356077413879824],[-72.47871565872217,42.356822670587604],[-72.47989583068919,42.35739349953569],[-72.47912335449298,42.35866198974499],[-72.47832942062406,42.35889982880852],[-72.4778788095096,42.360089

In [4]:
def create_features(data):
    output = []
    record = ast.literal_eval(data[0])

    for i in record:
        geometry = ast.literal_eval(i['area_geometry'])
        for point in geometry['coordinates']:
            result = {
                "type": "Feature",
                "geometry": geometry,
                "properties": {
                    "area" : i['area'], 
                    "id" : i['id'], 
                    "fac_name" : data[1],
                    "fac_state" : data[2],
                    "fac_street" : data[3],
                    "task_id" : data[4],
                    "task_user_id" : data[5],
                    "task_runinfo__skip" : data[6],
                    "task_runinfo__certain" : data[7],
                    "task_run__user_ip" : data[8],
                    "task_run__id" : data[9],
                    "task_run__created" : data[10],
                    "task_run__finish_time" : data[11],
                }
            }
            output.append(result)
            
    return output                

In [352]:
def create_geojson(data):
    #a stupid hack - zip all of these lists and pass them
    # into the create_feature function as variable temp
    
    data = data[data['task_runinfo__area'] != '0']
    
    temp = zip(
                list(data['task_runinfo__area']),
                list(data['task_runinfo__Facility_Name']),
                list(data['task_runinfo__Facility_State']),
                list(data['task_runinfo__Facility_Street']),
                list(data['task_run__task_id']),
                list(data['task_run__user_id']),
                list(data['task_runinfo__skip']),
                list(data['task_runinfo__certain']),
                list(data['task_run__user_ip']),
                list(data['task_run__id']),
                list(data['task_run__created']),
                list(data['task_run__finish_time']),
              )

    features = map(create_features, temp)

    # flatten the list
    features = [item for sublist in features for item in sublist]

    # remove entries without a geometry - no geometry causes import to fail
    features = filter(lambda x: len(x['geometry']['coordinates'][0]) > 2, features)
    features = filter(lambda x: x['properties']['area'] != 0, features)    

    # turn to string
    geojson = json.dumps({
        "type": "FeatureCollection",
        "features": features,
    })

    return geojson

In [353]:
geojson = create_geojson(data)

In [356]:
def writeFile(geojson):
    print "Writing geojson file..."
    outputFile = open('../landfill_geojson.geojson', 'wb')
    outputFile.write(geojson)
    outputFile.close()

In [357]:
writeFile(geojson)

Writing geojson file...
